In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_pickle('../input/feats/df_trn_feat0.pkl')
df_test  = pd.read_pickle('../input/feats/df_test_feat0.pkl')

In [3]:
from sklearn.model_selection import StratifiedKFold, KFold

In [19]:
data = df_train.iloc[:1000,:]
target='deal_probability'
feature ='param_1'
nfolds = 5
n_inner_folds=4

impact_coded = pd.Series()
df_new = data.copy()
oof_default_mean = data[target].mean()
kf = KFold(n_splits=nfolds, shuffle=True)
oof_mean_cv = pd.DataFrame()
split =0

for infold, oof in kf.split(data[feature]):    
    
    impact_coded_cv = pd.Series()
    kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
    inner_split = 0
    inner_oof_mean_cv = pd.DataFrame()
    oof_default_inner_mean = data.iloc[infold][target].mean()
    
    
    for infold_inner_no, (infold_inner, oof_inner) in enumerate(kf_inner.split(data.iloc[infold])):
        # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
        oof_mean = data.iloc[infold_inner].groupby(feature)[target].mean() ## oof mean comes from infold_inner
        impact_coded_cv_tmp = data.iloc[infold].copy() ## map to 
        impact_coded_cv_tmp = impact_coded_cv_tmp[feature].map(oof_mean).fillna(oof_default_inner_mean) 
        impact_coded_cv = impact_coded_cv.append(impact_coded_cv_tmp)
        # Also populate mapping (this has all group -> mean for all inner CV folds)        
        
        inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
        inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
        inner_split += 1
    
    oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
    oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
    split += 1
    
    oof_mean_cv_tmp = oof_mean_cv.mean(axis=1)
    
    impact_coded_tmp = data.iloc[oof].copy()
    impact_coded_tmp = impact_coded_tmp[feature].map(oof_mean_cv_tmp).fillna(oof_default_mean)
    impact_coded = impact_coded.append(impact_coded_tmp)
    
#     impact_coded = impact_coded.append(data.iloc[oof].apply(
#                 lambda x: inner_oof_mean_cv.loc[x[feature]].mean()
#                           if x[feature] in inner_oof_mean_cv.index
#                           else oof_default_mean
#                 , axis=1))


In [20]:
impact_coded.shape

(1000,)

In [9]:
impact_coded_tmp.shape

(200,)

In [103]:
oof

array([   0,   11,   20, ..., 9985, 9996, 9997])

In [98]:
inner_oof_mean_cv.shape

(371, 4)

In [21]:
np.random.seed(13)

def impact_coding(data, feature, target='y'):
    '''
    In this implementation we get the values and the dictionary as two different steps.
    This is just because initially we were ignoring the dictionary as a result variable.
    
    In this implementation the KFolds use shuffling. If you want reproducibility the cv 
    could be moved to a parameter.
    '''
    n_folds = 5
    n_inner_folds = 4
    impact_coded = pd.Series()
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    kf = KFold(n_splits=n_folds, shuffle=True)
    oof_mean_cv = pd.DataFrame()
    split = 0
    for infold, oof in kf.split(data[feature]):
        impact_coded_cv = pd.Series()
        kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
        inner_split = 0
        inner_oof_mean_cv = pd.DataFrame()
        oof_default_inner_mean = data.iloc[infold][target].mean()
        for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
            # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
            oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
            impact_coded_cv = impact_coded_cv.append(data.iloc[infold].apply(
                        lambda x: oof_mean[x[feature]]
                                  if x[feature] in oof_mean.index
                                  else oof_default_inner_mean
                        , axis=1))

            # Also populate mapping (this has all group -> mean for all inner CV folds)
            inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
            inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
            inner_split += 1

        # Also populate mapping
        oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
        oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
        split += 1

        impact_coded = impact_coded.append(data.iloc[oof].apply(
                        lambda x: inner_oof_mean_cv.loc[x[feature]].mean()
                                  if x[feature] in inner_oof_mean_cv.index
                                  else oof_default_mean
                        , axis=1))

    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean

In [ ]:
def mean_cv_encode(data, feature, target='deal_probability'):
    
    nfolds = 5
    n_inner_folds=4
    
    oof_default_mean = data[target].mean()
    kf = KFold(n_splits=nfolds, shuffle=True)
    oof_mean_cv = pd.DataFrame()
    split =0
    for infold, oof in kf.split(data[feature]):
        impact_code_cv = pd.Series()
        kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
        
        for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
            oof_mean = data.iloc[infold_inner].groupby(feature)[target].mean()

In [22]:
df_small = df_train.iloc[:1000,]

In [23]:
mean_encode = impact_coding(df_small,'param_1',target='deal_probability')

In [29]:
impact_coded

8      0.045802
18     0.045802
21     0.045802
25     0.130216
31     0.349227
37     0.037077
38     0.131021
50     0.061277
54     0.037077
75     0.045802
79     0.053280
93     0.013161
102    0.198277
108    0.061277
110    0.573325
113    0.703348
114    0.450597
136    0.131021
145    0.061277
158    0.045802
163    0.045802
166    0.045802
167    0.399621
168    0.061485
172    0.061485
173    0.388086
176    0.241406
179    0.399621
183    0.045802
189    0.131021
         ...   
893    0.061379
900    0.013819
901    0.045899
903    0.147510
910    0.061775
912    0.147510
914    0.404441
916    0.061775
919    0.036877
924    0.036877
932    0.352573
935    0.147510
938    0.147510
940    0.156395
941    0.147510
947    0.061775
952    0.147510
955    0.045899
960    0.045899
963    0.061379
964    0.045899
970    0.091017
971    0.091017
975    0.000000
976    0.244927
979    0.311741
983    0.061775
986    0.147510
992    0.045899
999    0.147510
Length: 1000, dtype: flo

In [28]:
mean_encode[0]

3      0.638682
4      0.331139
10     0.401652
15     0.045890
18     0.045890
21     0.045890
46     0.000000
51     0.131141
52     0.045890
58     0.057404
63     0.059728
65     0.392483
66     0.654167
70     0.059728
77     0.059728
78     0.131141
85     0.045890
87     0.149985
94     0.045890
98     0.331139
99     0.059728
106    0.131141
108    0.059728
111    0.045890
112    0.036260
114    0.449780
135    0.331139
165    0.045890
185    0.149985
189    0.131141
         ...   
866    0.045595
867    0.143410
880    0.045595
881    0.045595
888    0.394698
890    0.394698
893    0.061501
903    0.147510
909    0.394698
910    0.062076
915    0.139377
917    0.045595
922    0.143410
925    0.331337
941    0.143410
942    0.045595
947    0.062076
954    0.157032
955    0.045595
962    0.255381
966    0.331337
967    0.061501
969    0.143410
973    0.234328
977    0.130921
984    0.045595
988    0.061501
995    0.164297
996    0.319942
998    0.331337
Length: 1000, dtype: flo